In [4]:
#!pip install datasets pillow huggingface_hub


In [5]:
from google.colab import drive
drive.mount('/content/drive')
path='drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


### Reading Data From Drive

# Prediction - VIT

In [33]:
from datasets import load_dataset


dataset = load_dataset("imagefolder", data_dir=path+'test_temp')['train']
dataset

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Dataset({
    features: ['image'],
    num_rows: 22
})

### Definding our labels

In [40]:
labels=['baked_potato',
 'baklava',
 'caesar_salad',
 'cheese_sandwich',
 'cheesecake',
 'chicken',
 'chicken_curry',
 'chocolate_cake',
 'donuts',
 'eggs',
 'falafel',
 'fish',
 'french_fries',
 'hamburger',
 'hot_dog',
 'ice_cream',
 'lasagna',
 'omelette',
 'pizza',
 'spaghetti',
 'steak',
 'sushi',]

#https://github.com/openai/CLIP/blob/main/data/prompts.md for food101
prompt=['a photo of '+label+', a type of' for label in labels]




### Prediction - openai/clip-vit-large-patch14

In [54]:
from transformers import pipeline
checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")
def find_predict(predictions):
  return [item[0]['label'] for item in predictions]

predictions = detector(dataset['image'], candidate_labels=prompt)
y_pred=find_predict(predictions)

In [55]:
def prompt_to_label(x):
  for i in range(len(prompt)):
    if x==prompt[i]:
      return labels[i]

### Prediction to DataFrame

In [61]:
import os
import pandas as pd
Files_Name = os.listdir(path+'test_temp')
q1_submission_vit=pd.DataFrame(np.array([Files_Name ,[prompt_to_label(item) for item in y_pred]]).T,columns=['name','predicted'])
q1_submission_vit

,name,predicted
0,0a2f42a1-3b5.jpg,sushi
1,0a3ebd67-e44.jpg,steak
2,0a71fc11-924.jpg,spaghetti
3,0a24d7d4-ef5.jpg,eggs
4,0a305d4d-5f3.jpg,chicken
5,0a4ce48c-848.jpg,french_fries
6,0a4b6ec9-9c7.jpg,cheese_sandwich
7,0a5c9e70-66b.jpg,omelette
8,0a6bae39-72d.jpg,baked_potato
9,0a1a6049-7f1.jpg,cheesecake


### Saving Prediction

In [62]:
q1_submission_vit.to_csv('q1_submission_vit.csv', index=False)

# Prediction ResNet

In [79]:
import tensorflow as tf

Resnet=tf.keras.models.load_model(path+'image_classification.keras')

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': 'keras.layers', 'class_name': 'ZeroPadding2D', 'config': {'name': 'conv1_pad', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'padding': [[3, 3], [3, 3]], 'data_format': 'channels_last'}, 'registered_name': None, 'build_config': {'input_shape': [None, 224, 224, 3]}, 'name': 'conv1_pad', 'inbound_nodes': [{'args': [{'class_name': '__keras_tensor__', 'config': {'shape': [None, 224, 224, 3]